In [1]:
import pandas as pd
from fuzzywuzzy import fuzz
from collections import Counter

# Load the Excel file
df = pd.read_excel("neudata.xlsx")

# Step 1: Clean and standardize for fuzzy matching
df['HOLDER'] = df['HOLDER'].astype(str)
df['HOLDER_clean'] = df['HOLDER'].str.lower().str.strip()
df['HOLDER_standardized'] = df['HOLDER_clean']

# Initialize an empty list to hold the clusters of similar values
clusters = []

threshold = 80

# Step 2: Iterate through the rows and perform fuzzy matching
for i, val_i in enumerate(df['HOLDER_clean']):
    matched = False
    for cluster in clusters:
        for val_j in cluster:
            score = fuzz.token_sort_ratio(val_i, val_j)
            if score >= threshold:
                # Add val_i to the current cluster and mark as matched
                cluster.append(val_i)
                matched = True
                break
        if matched:
            break
    
    if not matched:
        # If no match was found, create a new cluster
        clusters.append([val_i])

# Step 3: Create a function to count the most frequent value in each cluster
def replace_with_most_frequent(cluster):
    # Count frequency of each value in the cluster
    frequency = Counter(cluster)
    
    # Get the most frequent value
    most_frequent_value = frequency.most_common(1)[0][0]
    
    # Replace all values in the cluster with the most frequent value
    return most_frequent_value

# Step 4: Create a new column with the most frequent value in each cluster
df['HOLDER_most_frequent'] = df['HOLDER_clean'].apply(
    lambda x: next(
        (replace_with_most_frequent(cluster) for cluster in clusters if x in cluster),
        x  # If not in any cluster, keep the original value
    )
)

# Step 5: Verify the result
print(df[['HOLDER', 'HOLDER_most_frequent']].head(300))

                                                HOLDER  \
0                             HEXCEL CHEMICAL PRODUCTS   
1                                     ROHM AND HAAS CO   
2                                          MONSANTO CO   
3                            NATIONAL RESEARCH COUNCIL   
4                                     WF STRAUB AND CO   
..                                                 ...   
295                 EMERY PERSONAL CARE PRODUCTS GROUP   
296                             CHAS PFIZER AND CO INC   
297                                   ICI AMERICAS INC   
298      LEDERLE LABORATORIES DIV AMERICAN CYANAMID CO   
299  RHONE POULENC PHARMACEUTICALS DIV RHONE POULEN...   

                              HOLDER_most_frequent  
0                         hexcel chemical products  
1                                 rohm and haas co  
2                                      monsanto co  
3                        national research council  
4                                 wf s

In [4]:
import spacy

# Load a pre-trained model
nlp = spacy.load("en_core_web_md")

def get_similarity(name1, name2):
    doc1 = nlp(name1)
    doc2 = nlp(name2)
    return doc1.similarity(doc2)

# Example usage
get_similarity("Pfizer", "Pfirzer")  # Likely to return a high similarity score


/var/folders/9p/5bcmz2c91nl9_sttnd189drw0000gn/T/ipykernel_49419/2835055086.py:9: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  return doc1.similarity(doc2)


0.0

In [7]:
import textdistance

def compare_strings(str1, str2):
    # Example using Jaro-Winkler distance
    return textdistance.jaro_winkler(str1, str2)

compare_strings('hexcel chemical products', 'american home products')  # Returns similarity score between 0 and 1


0.7289809863339275